In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os
import pandas as pd
from googleapiclient.http import MediaFileUpload

In [2]:
LOCAL_FOLDER = r"C:\Users\DELL\Desktop\vocuhers"
DRIVE_FOLDER_ID = "19S1KlWBYKee7auPpg10vviewQJpMqn6X"
SCOPES = ['https://www.googleapis.com/auth/drive']
CREDENTIALS_FILE = r"C:\Users\DELL\Desktop\voucher\key.json"
TOKEN_FILE = "token.pickle"

In [3]:
FOLDER_PATH = r"C:\Users\DELL\Desktop\vocuhers"
SCOPES = ['https://www.googleapis.com/auth/drive.file']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('key.json,SCOPES')
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

drive_service = build('drive', 'v3', credentials=creds)

In [4]:
for filename in os.listdir(LOCAL_FOLDER):
    file_path = os.path.join(LOCAL_FOLDER, filename)
    print(file_path)

C:\Users\DELL\Desktop\vocuhers\img_1.jpg
C:\Users\DELL\Desktop\vocuhers\img_2.jpg
C:\Users\DELL\Desktop\vocuhers\img_3.jpg
C:\Users\DELL\Desktop\vocuhers\voucher_image.jpg


In [5]:
for filename in os.listdir(LOCAL_FOLDER):
    file_path = os.path.join(LOCAL_FOLDER, filename)
    filename = os.path.basename(file_path)
    print(filename) 

img_1.jpg
img_2.jpg
img_3.jpg
voucher_image.jpg


In [6]:
for filename in os.listdir(LOCAL_FOLDER):
    file_path = os.path.join(LOCAL_FOLDER, filename)
    file_metadata = {'name': os.path.basename(file_path),'parents': [DRIVE_FOLDER_ID]}
    print(file_metadata)

{'name': 'img_1.jpg', 'parents': ['19S1KlWBYKee7auPpg10vviewQJpMqn6X']}
{'name': 'img_2.jpg', 'parents': ['19S1KlWBYKee7auPpg10vviewQJpMqn6X']}
{'name': 'img_3.jpg', 'parents': ['19S1KlWBYKee7auPpg10vviewQJpMqn6X']}
{'name': 'voucher_image.jpg', 'parents': ['19S1KlWBYKee7auPpg10vviewQJpMqn6X']}


In [7]:
def upload_file(file_path):
    file_metadata = {'name': os.path.basename(file_path),'parents': [DRIVE_FOLDER_ID]}
    media = MediaFileUpload(file_path, resumable=True)
    
    # Upload file
    uploaded_file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    
    # Make file publicly readable
    drive_service.permissions().create(
        fileId=uploaded_file.get('id'),
        body={'type': 'anyone', 'role': 'reader'}
    ).execute()
    
    # Generate direct link
    direct_link = f"https://drive.google.com/uc?export=view&id={uploaded_file.get('id')}"
    return direct_link

In [8]:
links = []
for file_name in os.listdir(FOLDER_PATH):
    file_path = os.path.join(FOLDER_PATH, file_name)
    if os.path.isfile(file_path):
        print(f"Uploading: {file_name}")
        link = upload_file(file_path)
        links.append({'file_name': file_name, 'link': link})
        print(f"✅ Uploaded: {file_name} → {link}")

Uploading: img_1.jpg
✅ Uploaded: img_1.jpg → https://drive.google.com/uc?export=view&id=1i4T0IB1_YWrQhANJ7SZTIupxOubYfT37
Uploading: img_2.jpg
✅ Uploaded: img_2.jpg → https://drive.google.com/uc?export=view&id=1phWG6wi8VRzAfyqXnhiB1uxA0wuXwKxf
Uploading: img_3.jpg
✅ Uploaded: img_3.jpg → https://drive.google.com/uc?export=view&id=1fcqDWrZomRbpGDVc_HYwnRBkXxAsX7mc
Uploading: voucher_image.jpg
✅ Uploaded: voucher_image.jpg → https://drive.google.com/uc?export=view&id=1WJQLARasN7bo69ImkjXSDP-6tUqmBLsN


In [9]:
df = pd.DataFrame(links)
df.to_csv("uploaded_voucher_links.csv", index=False)
print("✅ All links saved to uploaded_voucher_links.csv")

✅ All links saved to uploaded_voucher_links.csv
